<a href="#1-Accuracy" style="margin-left: 0px;">1 Accuracy</a>  
<a href="#2-Precision" style="margin-left: 0px;">2 Precision</a>    
<a href="#3-Recall" style="margin-left: 0px;">3 Recall</a>   
<a href="#4-F1-Score" style="margin-left: 0px;">4 F1 Score</a>    
<a href="#5-Fβ-Score" style="margin-left: 0px;">5 Fβ Score</a>   
<a href="#6-AUC-PR" style="margin-left: 0px;">6 AUC-PR</a>   
<a href="#7-AUC-ROC" style="margin-left: 0px;">7 AUC-ROC</a>   

In [1]:
import pandas as pd
import numpy as np

from category_encoders import TargetEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import precision_score, recall_score, auc, f1_score, fbeta_score, roc_auc_score, average_precision_score, accuracy_score, precision_recall_curve

| №  | Название колонки | Тип данных       | Описание |
|----|------------------|------------------|----------|
| 1  | `PassengerId`    | `int`            | Уникальный ID пассажира |
| 2  | `Survived`       | `int` (0/1)      | Выжил (1) или нет (0) — **таргет для предсказания** |
| 3  | `Pclass`         | `int` (1-3)      | Класс билета:<br>• 1 = 1-й класс<br>• 2 = 2-й класс<br>• 3 = 3-й класс |
| 4  | `Name`           | `string`         | Имя пассажира (включая титул, например, "Mr.", "Miss") |
| 5  | `Sex`            | `string`         | Пол (`"male"` или `"female"`) |
| 6  | `Age`            | `float`          | Возраст (есть пропуски — `NaN`) |
| 7  | `SibSp`          | `int`            | Количество siblings (братья/сёстры) + spouses (супруги) на борту |
| 8  | `Parch`          | `int`            | Количество parents (родители) + children (дети) на борту |
| 9  | `Ticket`         | `string`         | Номер билета (может содержать буквы и цифры) |
| 10 | `Fare`           | `float`          | Стоимость билета |
| 11 | `Cabin`          | `string`         | Номер каюты (много пропусков — `NaN`) |
| 12 | `Embarked`       | `string`         | Порт посадки:<br>• `C` = Cherbourg<br>• `Q` = Queenstown<br>• `S` = Southampton |

In [2]:
data = pd.read_csv(r"../00 Data/titanic.csv")

print(data.shape)
data.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# Замена пропусков в Age медианным значением
data['Age'] = data['Age'].fillna(data['Age'].median())

# Замена пропусков в Embarked модой
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

# Семейный размер
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

# Одиночка (1, если FamilySize = 1)
data['IsAlone'] = (data['FamilySize'] == 1).astype(int)

# Encoder
encoder = TargetEncoder(cols=['Sex', 'Embarked', 'FamilySize'])

data[['Sex', 'Embarked', 'FamilySize']] = encoder.fit_transform(
    data[['Sex', 'Embarked', 'FamilySize']], 
    data['Survived']  # Целевая переменная
)

In [4]:
# StandardScaler для всех признаков
features = data.columns.drop('Survived')
data[features] = StandardScaler().fit_transform(data[features])

In [5]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,0,0.827377,-0.737695,-0.565736,0.432793,-0.473674,-0.502445,-0.539973,1.290322,-1.231645
1,1,-1.566107,1.355574,0.663861,0.432793,-0.473674,0.786845,2.044556,1.290322,-1.231645
2,1,0.827377,1.355574,-0.258337,-0.474545,-0.473674,-0.488854,-0.539973,-0.687981,0.811922
3,1,-1.566107,1.355574,0.433312,0.432793,-0.473674,0.420730,-0.539973,1.290322,-1.231645
4,0,0.827377,-0.737695,0.433312,-0.474545,-0.473674,-0.486337,-0.539973,-0.687981,0.811922


In [6]:
logreg = LogisticRegression(max_iter=1000)  # Увеличиваем max_iter, если не сходится 
logreg.fit(data.iloc[:, 1:].values, data.iloc[:, 0].values)

LogisticRegression(max_iter=1000)

In [7]:
y_pred = logreg.predict(data.iloc[:, 1:].values)
y_proba = logreg.predict_proba(data.iloc[:, 1:].values)[:,1]

|   | y = 1 | y = 0 |
| ------------- | ------------- | ------------- |
| $\hat{y}=1$  | TP  | FP  |
| $\hat{y}=0$  | FN  | TN  |  

### 1 Accuracy

#### Подаем в качестве предсказаний 0 и 1  

**Accuracy** — это доля правильных предсказаний среди всех сделанных. Показывает, насколько часто модель классифицирует объекты верно.


$$
Accuracy = \frac{TP + TN}{TP + TN + FP + FN}
$$

Где:  
- **TP** — True Positive (истинно положительные)  
- **TN** — True Negative (истинно отрицательные)  
- **FP** — False Positive (ложноположительные)  
- **FN** — False Negative (ложноотрицательные)

**Пример интерпретации:**  
Если Accuracy = 0.92, значит 92% всех предсказаний оказались верными.

In [8]:
def my_accuracy(y_true, y_pred):
    return np.mean(np.array(y_true) == np.array(y_pred))

In [9]:
print(my_accuracy(data.iloc[:, 0].values, y_pred))
print(accuracy_score(data.iloc[:, 0].values, y_pred))

0.8114478114478114
0.8114478114478114


### 2 Precision

$\hat{y}$ - предсказания модели  
$y$ - фактические результаты  

#### Подаем в качестве предсказаний 0 и 1   

$$
\text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}}
$$
Где:
- **TP**: True Positives (истинно положительные)
- **FP**: False Positives (ложно положительные)

In [10]:
def my_precision(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    TP = sum((y_true == 1) & (y_pred == 1))  # True Positives
    FP = sum((y_true == 0) & (y_pred == 1))  # False Positives
    
    if (TP + FP) == 0:
        return 0
    
    return TP / (TP + FP)

In [11]:
print(my_precision(data.iloc[:, 0].values,y_pred))
print(precision_score(data.iloc[:, 0].values,y_pred))

0.7788461538461539
0.7788461538461539


### 3 Recall

#### Подаем в качестве предсказаний 0 и 1    

$$
\text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}
$$
Где:
- **TP**: True Positives (истинно положительные)
- **FN**: False Negatives (ложно отрицательные)

In [12]:
def my_recall(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    TP = sum((y_true == 1) & (y_pred == 1))  # True Positives
    FN = sum((y_true == 1) & (y_pred == 0))  # False Negatives
    
    if (TP + FN) == 0:
        return 0
    
    return TP / (TP + FN)

In [13]:
print(my_recall(data.iloc[:, 0].values,y_pred))
print(recall_score(data.iloc[:, 0].values,y_pred))

0.7105263157894737
0.7105263157894737


### 4 F1 Score

#### Подаем в качестве предсказаний 0 и 1  

Это гармоническое среднее между Precision и Recall.

$$
F_1 = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}} = \frac{2TP}{2TP + FP + FN}
$$

- **F₁** — это среднее *гармоническое* между Precision и Recall.  
- Даёт больший вес низким значениям: если Precision или Recall близок к 0, F₁ тоже будет близок к 0.  
- Идеально: **F₁ = 1** (когда Precision и Recall = 1).  
- Худший случай: **F₁ = 0** (если TP = 0).

In [14]:
def my_f1(y_true, y_pred):
    precision = my_precision(y_true, y_pred)
    recall = my_recall(y_true, y_pred)
    
    if precision+recall==0:
        return 0
    
    return 2*(precision*recall)/(precision+recall)

In [15]:
print(my_f1(data.iloc[:, 0].values,y_pred))
print(f1_score(data.iloc[:, 0].values,y_pred))

0.7431192660550459
0.7431192660550459


### 5 Fβ Score

#### Подаем в качестве предсказаний 0 и 1

$$
F_\beta = (1 + \beta^2) \cdot \frac{\text{Precision} \cdot \text{Recall}}{(\beta^2 \cdot \text{Precision}) + \text{Recall}}
$$

- **β** — параметр, определяющий важность Recall относительно Precision:
  - **β > 1**: Больший вес Recall (например, важно минимизировать FN).  
  - **β < 1**: Больший вес Precision (например, важно минимизировать FP).  
  - **β = 1**: Эквивалентно F₁-score.  
- Примеры:
  - **F₂-score** (β=2): Recall в 2 раза важнее Precision.  
  - **F₀.₅-score** (β=0.5): Precision в 2 раза важнее Recall.

In [16]:
def my_f_b(y_true, y_pred, beta):
    precision = my_precision(y_true, y_pred)
    recall = my_recall(y_true, y_pred)
    
    if precision+recall==0:
        return 0
    
    return ((1+beta**2)*(precision*recall))/((beta**2)*precision+recall)

In [17]:
print(my_f_b(data.iloc[:, 0].values,y_pred,beta=2))
print(fbeta_score(data.iloc[:, 0].values,y_pred,beta=2))

0.7232142857142857
0.7232142857142857


### 6 AUC-PR

#### AUC-PR (Площадь под кривой Precision-Recall)

$$
\text{AUC-PR} = \int_0^1 \text{Precision}(r) \, dr
$$
Где:
- **Precision(r)**: Точность при определённом уровне Recall (r).
- AUC-PR измеряет площадь под кривой Precision-Recall.

#### Матрица предсказаний

**y_pred_matrix = (prob[:, None] >= thresholds).astype(int)**  
    
prob = np.array([0.9, 0.7, 0.2, 0.4])  
thresholds = np.array([0.5, 0.8])  

то означает [:, None]?
1) Возьмите все элементы вдоль первой (нулевой) оси (то есть строки).  
2) None: Добавьте новую ось (новое измерение) в конце.  

prob[:, None] =    
array([[0.9],  
       [0.7],  
       [0.2],  
       [0.4]])  

**y_pred_matrix = (prob[:, None] >= thresholds).astype(int)**  

#### Результат:  
y_pred_matrix = 
[[ True  True]  
 [ True False]  
 [False False]  
 [False False]]  
 
 #### Столбец это значения y_pred для порога, разные столбцы это разные пороги

#### Суммируем по столбцам и получаем TP для каждого порога
**TP = np.sum((y_pred_matrix == 1) & (y_true[:, None] == 1), axis=0)**

In [18]:
def my_auc_pr(y_true, prob):
    # Округляем вероятности, чтобы уменьшить количество порогов
    prob = np.round(prob, decimals=6)
    y_true = np.array(y_true)
    
    # Выбираем равномерно распределенные пороги
    # thresholds = np.linspace(0, 1, 100)
    thresholds = np.unique(prob)

    
    # Матрица предсказаний
    y_pred_matrix = (prob[:, None] >= thresholds).astype(int)
    
    # TP, FP, FN
    TP = np.sum((y_pred_matrix == 1) & (y_true[:, None] == 1), axis=0)
    FP = np.sum((y_pred_matrix == 1) & (y_true[:, None] == 0), axis=0)
    FN = np.sum((y_pred_matrix == 0) & (y_true[:, None] == 1), axis=0)
    
    # Precision и Recall с безопасным делением
    precision = np.divide(TP, TP + FP, out=np.zeros_like(TP, dtype=float), where=(TP + FP) != 0)
    recall = np.divide(TP, TP + FN, out=np.zeros_like(TP, dtype=float), where=(TP + FN) != 0)
    
    # AUC
    return auc(recall, precision)

In [19]:
print(my_auc_pr(data.iloc[:, 0].values,y_proba))
precision, recall, thresholds = precision_recall_curve(data.iloc[:, 0].values,y_proba)
print(auc(recall, precision))

0.8224754177105095
0.8224754177105095


### 7 AUC-ROC 

ROC-кривая: строится по TPR (True Positive Rate) и FPR (False Positive Rate).

AUC-ROC показывает, насколько хорошо модель разделяет положительные и отрицательные классы.

**Как строится ROC-кривая:**  

TPR(true positive rate) = $\frac{TP}{TP+FN}$ (x-axis - ось абсцисс x)  
FPR(false positive rate) = $\frac{FP}{FP+TN}$ (y-axis - ось ординат x)  


1. Сортируем объекты по убыванию вероятности класса "1".  
2. Начинаем с точки (0, 0).  
3. Для каждого объекта:  
   - Если истинный класс = 1 → увеличиваем TPR (двигаемся вверх).  
   - Если истинный класс = 0 → увеличиваем FPR (двигаемся вправо).  
4. AUC-ROC — площадь под получившейся кривой.  

In [20]:
def my_auc_roc(y_true, prob):
    # Округляем вероятности, чтобы избежать лишних порогов
    prob = np.round(prob, decimals=6)
    y_true = np.array(y_true)

    # Уникальные значения вероятностей — будут нашими порогами
    thresholds = np.sort(np.unique(prob))[::-1]  # от большего к меньшему

    # Матрица предсказаний
    y_pred_matrix = (prob[:, None] >= thresholds).astype(int)

    # TP, FP, TN, FN
    TP = np.sum((y_pred_matrix == 1) & (y_true[:, None] == 1), axis=0)
    FP = np.sum((y_pred_matrix == 1) & (y_true[:, None] == 0), axis=0)
    TN = np.sum((y_pred_matrix == 0) & (y_true[:, None] == 0), axis=0)
    FN = np.sum((y_pred_matrix == 0) & (y_true[:, None] == 1), axis=0)

    # TPR (Recall) и FPR с безопасным делением
    TPR = np.divide(TP, TP + FN, out=np.zeros_like(TP, dtype=float), where=(TP + FN) != 0)
    FPR = np.divide(FP, FP + TN, out=np.zeros_like(FP, dtype=float), where=(FP + TN) != 0)

    # Возвращаем площадь под кривой (x=FPR, y=TPR)
    return auc(FPR, TPR)

In [21]:
print(my_auc_roc(data.iloc[:, 0].values,y_proba))
print(roc_auc_score(data.iloc[:, 0].values, y_proba))

0.8644185600613556
0.8644185600613554


#### Результаты схожие